## Multi pendulum tutorial
---

### Initial setup

Some initial imports to get output looking right in the notebook.

In [ ]:
# only really needed for pretty printing
import sympy as sp

# for setting times, initial conditions
import numpy as np

# pretty printing of mathematical expressions
sp.init_printing()

# to display animations
from IPython.display import HTML

# to display plots in the notebook
%matplotlib inline

Import the multipendulum module.

In [ ]:
import multipendulum as mp

---

### Create and configure a MultiPendulum object

Nearly all of the work is done (for now) by the `MultiPendulum` object. In the near future, plots and other diagnostics will likely be moved out.

When you create a `MultiPendulum`, you need to tell it how many segments it has. 

In [ ]:
double = mp.MultiPendulum(2)

You can set the lengths of the segments, or the masses, or both. By default, the lengths are each $1/n$ (for $n$ segments) and the masses are each 1.

In [ ]:
double.set_lengths((0.4, 0.6))
double.set_masses((1.5, 3.0))

With the lengths and masses set, you can look at the (squared, angular) linear eigenfrequencies

In [ ]:
double.D

... and the (non normalized) linear eigenmodes

In [ ]:
double.S

If you change the lengths or masses, the eigenmodes and eigenfrequencies are recalculated.

In [ ]:
double.set_lengths((0.2, 0.8))
double.D

In [ ]:
double.S

---
### Integrating

For time integration, the procedure is:
1. set initial conditions
2. set the times for integration
3. perform the integration

#### Initial conditions

Initial conditions are specified via the `set_initial_conditions` method.

In [ ]:
help(mp.MultiPendulum.set_initial_conditions)

In [ ]:
double.set_initial_conditions((0,60), (180, 0), degrees=True)
double.y0

In [ ]:
double.set_initial_conditions(1, 0, eigenmodes=True)
double.y0

#### Integration times

The default integration is 100s long with a timestep of 0.01s, for a total of 10,000 samples.

In [ ]:
double.times

Any reasonable duration and timestep size should work, however, and can be specified by creating an array of times:

In [ ]:
times = np.linspace(0, 3000, 1000000)
print("times has {} samples between {} and {}.".format(len(times), times[0], times[-1]))

More samples will of course mean longer (wall clock) integration times.

#### Carrying out the integration

In [ ]:
help(mp.MultiPendulum.integrate)

In [ ]:
times = np.linspace(0, 30, 1000)
double.integrate(times)

There's really nothing to see for the results of the integration without using plots of some kind.

---

### Plots

Time evolution can be viewed in a few different ways. For the totally qualitative, you can use an animation. These are good for producing intuition, but typically aren't as precise as other plots (unless carefully constructed).

In [ ]:
times = np.linspace(0, 30, 1000)
double.animate(times)
HTML(double.anim.to_html5_video())

We can increase the precision of our plots a bit by going to timeseries. Since we probably want to see all coordinates and velocities, we have a function that produces a single figure (with subfigures) which captures all of this information.

In [ ]:
fig = double.time_series_plots()

Another way to display this information is by plotting a slice of the attractor in two of the phase space dimensions. The natural approach is to take a coordinate and its associated velocity, so that's what I've done here.

In [ ]:
fig = double.phase_plots()

Both of these plotting routines can also show the time behavior of the linear eigenmodes, rather than the individual pendulums.

In [ ]:
fig = double.phase_plots(eigenmodes=True)

In [ ]:
fig = double.time_series_plots(eigenmodes=True)

Lastly (for now) we also want to have more detailed information about the frequencies present in a particular time series. We get this information from a frequency power spectrum (the square magnitude of the FFT of the time series).

In [ ]:
fig = double.powerspectrum()

Again, this also works for the projection onto the linear eigenmode basis.

In [ ]:
fig = double.powerspectrum(eigenmodes=True)

---

### Saving output
You will want to save output for presentations and to avoid duplicating effort. In the former case, you can get away with just saving pictures and animations, but in the latter case, you'll want to save the whole raw time series.  

#### Saving animations

The `anim` object has a method for writing to a file. It has a bewildering array of options. You should try to get away with using the defaults unless and until you make a movie with poor visual quality, at which time you can begin to tweak things.

In [ ]:
# this will save tut-temp.mp4 in the same folder as this notebook
double.anim.save("tut-temp.mp4")

For reference, here is the docstring:

In [ ]:
help(double.anim.save)

#### Saving figures

Figures can also be saved easily. All of the figure plotting methods return a figure object which has a `savefig` method. The only argument you need to supply is the file name.

In [ ]:
fig = double.powerspectrum(eigenmodes=True)

# should create 'tut-temp.png' in the folder containing this notebook.
fig.savefig('tut-temp.png')

There are, of course, other possible arguments.

In [ ]:
help(fig.savefig)

#### Saving raw data

In many cases, the data you generate with this model is pretty easy/quick to replicate; even with a million points in the time series, integrating the ODEs takes seconds rather than hours. We may get to a point, however, when that is not the case (if we move to larger numbers of links in the multipendulum, for example) and so it's useful to be able to save the results of a simulation for later analysis. We do that with the `serialize` method.

In [ ]:
help(double.serialize)

In [ ]:
# will create tut-temp.h5 in the folder containing this notebook.
double.serialize("tut-temp.h5")

Since this functionality is mostly future-proofing at this point, I haven't yet written functions to extract data from the archive.

---

### The todo list
There are a number of things still to add to this multipendulum object:

1. **Energetics.** We don't yet have any diagnostics for measuring or plotting energies. We will need them.
2. **Nonlinear Normal Modes.** To connect this with previous work.
3. **Liapunov exponents.** If we're interested in chaos, this has to be here.
4. **Poincare sections.** Also.
5. **Unit tests.** Right now the code is simple enough that I'm relatively confident there are no major bugs, but there ought to be a test suite to verify this.
6. **Further annotations on plots.** There is some additional information which should be included on the plots.
7. **Refactoring.** The plotting functions should probably be moved into their own module within the package. This will change the API slightly
8. **Physical bells and whistles.** In particular, I'd like to add the ability to have damping and driving.